# Differential Abundance with Linear Mixed Effects Modeling
### Cayla Mason

This notebook allows you to start from uncounted features and obtain differential abundance categorically OR over a continuous variable.<br><br>
General workflow:
1. Start with uncounted feature set, get feature counts per sample
2. Remove rare features
3. Zero imputation via Bayesian Multiplicative Replacement
4. CLR transform
5. Differential abundance

In [ ]:
library(nlme)
library(tidyverse)
library(magrittr)
library(multcomp)

Feature table format example

| Sample | Feature |
| --- | --- | 
| Sample_1 | Arthrobacter |
| Sample_1 | Akkermancia |
etc.

In [ ]:
# load feature table
# this should be pre-filtered according to quality (i.e. e-value or percent identity, etc.)
features <- read_csv('features.csv')
head(features)

In [ ]:
# get feature counts per sample
feature_counts <- features %>%  
    group_by(Sample) %>% 
    count(Feature) %>% 
    ungroup()

In [ ]:
# get sample counts per feature
sample_counts <- features %>% 
    dplyr::select(Sample, Feature) %>% 
    distinct() %>% 
    group_by(Feature) %>% 
    count() %>% 
    ungroup()

In [ ]:
# filter out rare features 
# threshold = 90%
# features present in less than 90% of samples are removed
n_samples <- features %>% dplyr::select(sample) %>% distinct() %>% nrow()
sample_counts %<>% filter(n >= floor(0.9*n_samples))
feature_counts %<>% filter(Feature %in% sample_counts$Feature)

In [ ]:
# place counts in wide format
feature_counts %<>% pivot_wider(names_from='Feature', values_from='n', values_fill=0)

In [ ]:
# zero imputation
if (any(feature_counts == 0)){
    mat <- feature_counts %>% dplyr::select(-Sample) %>% as.matrix()
    counts_no0 <- zCompositions::cmultRepl(mat, output = "p-counts", label = 0)
} else {
    counts_no0 <- feature_counts
}

In [ ]:
# CLR transform function
clr <- function(mtrx, na.rm = TRUE){
    sweep(log(mtrx), 1, rowMeans(log(mtrx)), "-")
}

In [ ]:
# CLR transformation
clr_counts <- as.matrix(counts_no0)
clr_counts <- as.data.frame(clr(clr_counts))
clr_counts$Sample <- feature_counts$Sample

# save for later
clr_counts %>% write_csv(paste(lubridate::today(),'CLR_features.csv', sep='_'))

In [ ]:
# code the features/column names for ease
Feature <- colnames(clr_counts)
code <- paste('Feature', seq(1,length(Features)), sep='_')
Feature_code <- data.frame(Feature, code)
Feature_code %>% write_csv(paste(lubridate::today(), 'Feature_Label_Encoding.csv', sep='_'))

# replace the labels
colnames(clr_counts) <- code

In [ ]:
# change data types
clr_counts %<>% mutate(across(where(is.character), as.factor)

In [ ]:
# load metadata
metadata <- read_csv('metadata.csv')

In [ ]:
metadata %<>% mutate(Sample = as.factor(Sample),  # change data types as needed
                     x.scaled = scale(x))  # scale and center all continuous variables of interest

## Differential Abundance by Categorical Variable

Note on linear mixed effects models:
Let's say you have mice and you are taking multiple samples from each mouse over time. If you have less than 5 time points, the time point should be considered a fixed effect. Otherwise, it should be considered a random effect.
- time points < 5:
    - fixed = feature ~ variable(s) of interest + time_point
    - random = ~ 1 | mouse_ID
- time points >=5:
    - fixed = feature ~ variable(s) of interest
    - random = ~ time_point | mouse_ID

In [ ]:
n_features <- clr_counts %>% dplyr::select(-Sample) %>% ncol()
clr_data <- clr_counts %>% left_join(metadata) %>% as.data.frame() %>% relocate(Sample)
H_null <- 'time_point'
H_alt <- 'categorical_variable + time_point'
rand <- 'mouse_ID'



# --------------------- Linear Mixed Effects Modeling ---------------------

t.start <- Sys.time()

# instantiate data frame to return results in
results <- data.frame(Feature = character(),
                      Main_p_value = double(),
                      Lowest_AIC = logical(),
                      Sig_to_H0 = logical(),
                      Main_Slope = double(),
                      Normal_Dist = logical(),
                      Equal_var = logical(),
                      Two_vs_One_coef = double(),     # you will need a column for each level within your  
                      Three_vs_One_coef = double(),   # categorical variable for both the slope (coef) and p-value
                      Three_vs_Two_coef = double(),   # here, there are 3 levels within the variable
                      Two_vs_One_p = double(),        # you will need to change it below as well***
                      Three_vs_One_p = double(),
                      Three_vs_Two_p = double(),
                      stringsAsFactors = FALSE)


# loop over features and get models!
for (i in 2:n_features){
    
    feature <- colnames(clr_data[i])
    fixed.null <- formula(paste(feature,'~',H_null))
    fixed.alt <- formula(paste(feature,'~',H_alt))
    rando.form <- formula(paste('~ 1|',rand, sep=''))
    
    tryCatch(
        # TRY BLOCK
        { 
            # fitting the linear mixed effects model
            # null model
            {null_model <- lme(data = clr_data,
                      fixed = fixed.null,
                      random = rando.form,
                      na.action=na.omit,
                      method='ML')}
            
            {alt_model <- lme(data = clr_data,
                     fixed = fixed.alt,
                     random = rando.form,
                     na.action=na.omit,
                     method='ML')}
        },
        # ERROR BLOCK
        error=function(e) {
            null_model <- NULL
            alt_model <- NULL
       })

    if(!is.null(alt_model)){
        # compare LMMs and get T/F lowest AIC
        comparison <- anova(null_model, alt_model)
        Lowest_AIC <- (which.min(comparison$AIC) == 2) 
        null_comp <- (comparison[2,9] <= 0.05)
        
        # get p-values 
        test <- car::Anova(alt_model, type = 2)
        main_p_val <- test[1,3]
        
        # get coefficients
        main_slope <- coef(alt_model)[1,2]
        
        # normality assessment
        normal <- (shapiro.test(resid(alt_model))$p.value > 0.05)
    
        # homogeneity of variance, H0: equal
        # https://ademos.people.uic.edu/Chapter18.html#62_assumption_2_homogeneity_of_variance
        clr_data$sq.resid <- abs(resid(alt_model))^2
        levene <- lm(sq.resid ~ Site, data = clr_data)
        equal_var <- (anova(levene)[1,5] > 0.05)
        
        # if main p-value is <= 0.05, perform multiple comparisons
        if(main_p_val<=0.05 & normal==TRUE & equal_var==TRUE){
            g <- glht(alt_model, linfct=mcp(Type = 'Tukey'))
            s <- summary(g)
            Two_One_coef <- unname(s$test$coefficients[1])       # *** add columns here if needed 
            Three_One_coef <- unname(s$test$coefficients[2])
            Three_Two_coef <- unname(s$test$coefficients[3])
            Two_One_p <- s$test$pvalues[1]                       # *** add columns here if needed
            Three_One_p <- s$test$pvalues[2]
            Three_Two_p <- s$test$pvalues[3]
        } else {
            Two_One_coef <- NA
            Three_One_coef <- NA
            Three_Two_coef <- NA
            Two_One_p <- NA
            Three_One_p <- NA
            Three_Two_p <- NA
        }
        
    } 
    
    # place all information into data frame
    df <- data.frame(feature,main_p_val, Lowest_AIC, null_comp, main_slope, normal, equal_var, 
                     Two_One_coef, Three_One_coef, Three_Two_coef,  # *** add columns here if needed
                     Two_One_p, Three_One_p, Three_Two_p)           # *** add columns here if needed
    rownames(df) <- NULL
    names(df) <- names(results)

    results %<>% bind_rows(df)
}

t.end <- Sys.time()
t.end - t.start

In [ ]:
# subset features that cannot be modeled using LMM without modification
# (i.e. they do not meet the assumptions)
non_norm <- results %>% filter(Normal_Dist == FALSE)
unequal_vars <- results %>% filter(Equal_var == FALSE)

In [ ]:
n_features <- nrow(non_norm)

# perform cube-root transformation to preserve negative values
clr_data <- clr_counts %>% 
    dplyr::select(Sample, all_of(non_norm$Feature)) %>% 
    mutate(across(all_of(non_norm$Feature), function(x) if_else(x<0, -1*(abs(x)^(1/3)), x^(1/3)))) %>% 
    left_join(metadata) %>% 
    as.data.frame() %>% 
    relocate(Sample)
                  
H_null <- 'time_point'
H_alt <- 'categorical_variable + time_point'
rand <- 'mouse_ID'


                  

# --------------------- Linear Mixed Effects Modeling ---------------------

t.start <- Sys.time()

# instantiate data frame to return results in
xformed <- data.frame(Feature = character(),
                      Main_p_value = double(),
                      Lowest_AIC = logical(),
                      Sig_to_H0 = logical(),
                      Main_Slope = double(),
                      Normal_Dist = logical(),
                      Equal_var = logical(),
                      Two_vs_One_coef = double(),     # you will need a column for each level within your  
                      Three_vs_One_coef = double(),   # categorical variable for both the slope (coef) and p-value
                      Three_vs_Two_coef = double(),   # here, there are 3 levels within the variable
                      Two_vs_One_p = double(),        # you will need to change it below as well***
                      Three_vs_One_p = double(),
                      Three_vs_Two_p = double(),
                      stringsAsFactors = FALSE)


# loop over features and get models!
for (i in 2:n_features){
    
    feature <- colnames(clr_data[i])
    fixed.null <- formula(paste(feature,'~',H_null))
    fixed.alt <- formula(paste(feature,'~',H_alt))
    rando.form <- formula(paste('~ 1|',rand, sep=''))
    
    tryCatch(
        # TRY BLOCK
        { 
            # fitting the linear mixed effects model
            # null model
            {null_model <- lme(data = clr_data,
                      fixed = fixed.null,
                      random = rando.form,
                      na.action=na.omit,
                      method='ML')}
            
            {alt_model <- lme(data = clr_data,
                     fixed = fixed.alt,
                     random = rando.form,
                     na.action=na.omit,
                     method='ML')}
        },
        # ERROR BLOCK
        error=function(e) {
            null_model <- NULL
            alt_model <- NULL
       })

    if(!is.null(alt_model)){
        # compare LMMs and get T/F lowest AIC
        comparison <- anova(null_model, alt_model)
        Lowest_AIC <- (which.min(comparison$AIC) == 2) 
        null_comp <- (comparison[2,9] <= 0.05)
        
        # get p-values 
        test <- car::Anova(alt_model, type = 2)
        main_p_val <- test[1,3]
        
        # get coefficients
        main_slope <- coef(alt_model)[1,2]
        
        # normality assessment
        normal <- (shapiro.test(resid(alt_model))$p.value > 0.05)
    
        # homogeneity of variance, H0: equal
        # https://ademos.people.uic.edu/Chapter18.html#62_assumption_2_homogeneity_of_variance
        clr_data$sq.resid <- abs(resid(alt_model))^2
        levene <- lm(sq.resid ~ Site, data = clr_data)
        equal_var <- (anova(levene)[1,5] > 0.05)
        
        # if main p-value is <= 0.05, perform multiple comparisons
        if(main_p_val<=0.05 & normal==TRUE & equal_var==TRUE){
            g <- glht(alt_model, linfct=mcp(Type = 'Tukey'))
            s <- summary(g)
            Two_One_coef <- unname(s$test$coefficients[1])       # *** add columns here if needed 
            Three_One_coef <- unname(s$test$coefficients[2])
            Three_Two_coef <- unname(s$test$coefficients[3])
            Two_One_p <- s$test$pvalues[1]                       # *** add columns here if needed
            Three_One_p <- s$test$pvalues[2]
            Three_Two_p <- s$test$pvalues[3]
        } else {
            Two_One_coef <- NA
            Three_One_coef <- NA
            Three_Two_coef <- NA
            Two_One_p <- NA
            Three_One_p <- NA
            Three_Two_p <- NA
        }
        
    } 
    
    # place all information into data frame
    df <- data.frame(feature,main_p_val, Lowest_AIC, null_comp, main_slope, normal, equal_var, 
                     Two_One_coef, Three_One_coef, Three_Two_coef,  # *** add columns here if needed
                     Two_One_p, Three_One_p, Three_Two_p)           # *** add columns here if needed
    rownames(df) <- NULL
    names(df) <- names(xformed)

    xformed %<>% bind_rows(df)
}

t.end <- Sys.time()
t.end - t.start

In [ ]:
n_features <- nrow(unequal.vars)

clr_data <- clr_counts %>% 
    dplyr::select(sample, all_of(unequal_vars$Feature)) %>% 
    left_join(metadata) %>% 
    as.data.frame() %>% 
    relocate(Sample)

H_null <- 'time_point'
H_alt <- 'categorical_variable + time_point'
rand <- 'mouse_ID'
                  
                  

# --------------------- Linear Mixed Effects Modeling ---------------------

t.start <- Sys.time()

# instantiate data frame to return results in
weighted <- data.frame(Feature = character(),
                      Main_p_value = double(),
                      Lowest_AIC = logical(),
                      Sig_to_H0 = logical(),
                      Main_Slope = double(),
                      Normal_Dist = logical(),
                      Equal_var = logical(),
                      Two_vs_One_coef = double(),     # you will need a column for each level within your  
                      Three_vs_One_coef = double(),   # categorical variable for both the slope (coef) and p-value
                      Three_vs_Two_coef = double(),   # here, there are 3 levels within the variable
                      Two_vs_One_p = double(),        # you will need to change it below as well***
                      Three_vs_One_p = double(),
                      Three_vs_Two_p = double(),
                      stringsAsFactors = FALSE)


# loop over features and get models!
for (i in 2:n_features){
    
    feature <- colnames(clr_data[i])
    fixed.null <- formula(paste(feature,'~',H_null))
    fixed.alt <- formula(paste(feature,'~',H_alt))
    rando.form <- formula(paste('~ 1|',rand, sep=''))
    weight <- formula(paste('~I(1/',feature,')'))
    
    tryCatch(
        # TRY BLOCK
        { 
            # fitting the linear mixed effects model
            # null model
            {null_model <- lme(data = clr_data,
                      fixed = fixed.null,
                      random = rando.form,
                      na.action=na.omit,
                      method='ML',
                      weights = weight)}
            
            {alt_model <- lme(data = clr_data,
                     fixed = fixed.alt,
                     random = rando.form,
                     na.action=na.omit,
                     method='ML',
                     weights = weight)}
        },
        # ERROR BLOCK
        error=function(e) {
            null_model <- NULL
            alt_model <- NULL
       })

    if(!is.null(alt_model)){
        # compare LMMs and get T/F lowest AIC
        comparison <- anova(null_model, alt_model)
        Lowest_AIC <- (which.min(comparison$AIC) == 2) 
        null_comp <- (comparison[2,9] <= 0.05)
        
        # get p-values 
        test <- car::Anova(alt_model, type = 2)
        main_p_val <- test[1,3]
        
        # get coefficients
        main_slope <- coef(alt_model)[1,2]
        
        # normality assessment
        normal <- (shapiro.test(resid(alt_model))$p.value > 0.05)
    
        # homogeneity of variance, H0: equal
        # https://ademos.people.uic.edu/Chapter18.html#62_assumption_2_homogeneity_of_variance
        clr_data$sq.resid <- abs(resid(alt_model))^2
        levene <- lm(sq.resid ~ Site, data = clr_data)
        equal_var <- (anova(levene)[1,5] > 0.05)
        
        # if main p-value is <= 0.05, perform multiple comparisons
        if(main_p_val<=0.05 & normal==TRUE & equal_var==TRUE){
            g <- glht(alt_model, linfct=mcp(Type = 'Tukey'))
            s <- summary(g)
            Two_One_coef <- unname(s$test$coefficients[1])       # *** add columns here if needed 
            Three_One_coef <- unname(s$test$coefficients[2])
            Three_Two_coef <- unname(s$test$coefficients[3])
            Two_One_p <- s$test$pvalues[1]                       # *** add columns here if needed
            Three_One_p <- s$test$pvalues[2]
            Three_Two_p <- s$test$pvalues[3]
        } else {
            Two_One_coef <- NA
            Three_One_coef <- NA
            Three_Two_coef <- NA
            Two_One_p <- NA
            Three_One_p <- NA
            Three_Two_p <- NA
        }
        
    } 
    
    # place all information into data frame
    df <- data.frame(feature,main_p_val, Lowest_AIC, null_comp, main_slope, normal, equal_var, 
                     Two_One_coef, Three_One_coef, Three_Two_coef,  # *** add columns here if needed
                     Two_One_p, Three_One_p, Three_Two_p)           # *** add columns here if needed
    rownames(df) <- NULL
    names(df) <- names(weighted)

    weighted %<>% bind_rows(df)
}

t.end <- Sys.time()
t.end - t.start

In [ ]:
results %<>% bind_rows(xformed, weighted)

lmms <- results %>% 
    filter(Lowest_AIC==T, Sig_to_H0==T, Normal_Dist==T, Equal_var==T) %>% 
    group_by(Feature) %>% 
    filter(Main_p_value == min(Main_p_value)) %>% 
    ungroup() %>% 
    mutate(main_p_adj = p.adjust(Main_p_value, method='fdr'), 
           covar_p_adj = p.adjust(Covar_p_value, method='fdr')) %>% 
    filter(main_p_adj <= 0.05)

In [ ]:
# save to file
lmms %>% write_csv(paste(lubridate::today(),'LMM_Categorical_Variable.csv', sep='_'))

### Level 1 v Level 3

In [ ]:
# What is changing the most? 
x <- lmms %>% 
    filter(Three_vs_One_p <= 0.05) %>% 
    arrange(Three_vs_One_coef) 

higher <- x %>% slice_head(n=10)
lower <- x %>% slice_tail(n=10)
most_diff <- bind_rows(higher, lower)

data <- clr_counts %>% 
    dplyr::select(Sample, all_of(most_diff$Feature)) %>% 
    pivot_longer(-Sample, names_to='code', values_to='n') %>% 
    left_join(most_diff, by=c('code' = 'Feature')) %>% 
    left_join(Feature_code) %>% 
    left_join(metadata)

In [ ]:
data %>% 
    filter(Type != 'Level 2') %>% 
    mutate(Feature = fct_reorder(Feature, Three_vs_One_coef)) %>% 
    ggplot(aes(mouse_ID, Feature)) +
    geom_tile(aes(fill = n)) +
    scale_fill_gradient2(name="CLR\nAbundance", low='#4E84C4', mid='#F4EDCA', high='#D16103') +
    theme_minimal(base_size=16) +
    labs(title = 'Significantly Changing Features',
         x = NULL,
         y = NULL) +
    theme(plot.title = element_text(hjust = 0.5),
          axis.text.x = element_blank(),
          panel.grid.major = element_blank(), 
          panel.grid.minor = element_blank()) +
    scale_x_discrete(limits=c('mouse_1','mouse_2','mouse_3','mouse_4')) # levels within mouse_ID in preferred order

In [ ]:
ggsave(filename=paste(lubridate::today(),'One v Three.pdf'),
       dpi='retina', device='pdf', bg='transparent')

## Differential Abundance by Continuous Variable

### Continuous Variable x

In [ ]:
n_features <- clr_counts %>% dplyr::select(-Sample) %>% ncol()
clr_data <- clr_counts %>% left_join(metadata) %>% as.data.frame() %>% relocate(Sample)
H_null <- 'time_point'
H_alt <- 'x.scaled + time_point'
rand <- 'mouse_ID'


# --------------------- Linear Mixed Effects Modeling ---------------------
# Using this method to assess differential abundance while maintaining power

t.start <- Sys.time()

# instantiate data frame to return results in
results <- data.frame(Feature = character(),
                      Main_p_value = double(),
                      Lowest_AIC = logical(),
                      Sig_to_H0 = logical(),
                      Main_Slope = double(),
                      Normal_Dist = logical(),
                      Equal_var = logical(),
                      stringsAsFactors = FALSE)


# loop over features and get models!
for (i in 2:n_features){
    
    feature <- colnames(clr_data[i])
    fixed.null <- formula(paste(feature,'~',H_null))
    fixed.alt <- formula(paste(feature,'~',H_alt))
    rando.form <- formula(paste('~ 1|',rand, sep=''))
    
    tryCatch(
        # TRY BLOCK
        { 
            # fitting the linear mixed effects model
            # null model
            {null_model <- lme(data = clr_data,
                      fixed = fixed.null,
                      random = rando.form,
                      na.action=na.omit,
                      method='ML')}
            
            {alt_model <- lme(data = clr_data,
                     fixed = fixed.alt,
                     random = rando.form,
                     na.action=na.omit,
                     method='ML')}
        },
        # ERROR BLOCK
        error=function(e) {
            null_model <- NULL
            alt_model <- NULL
       })

    if(!is.null(alt_model)){
        # compare LMMs and get T/F lowest AIC
        comparison <- anova(null_model, alt_model)
        Lowest_AIC <- (which.min(comparison$AIC) == 2) 
        null_comp <- (comparison[2,9] <= 0.05)
        
        # get p-values 
        test <- car::Anova(alt_model, type = 2)
        main_p_val <- test[1,3]
        
        # get coefficients
        main_slope <- coef(alt_model)[1,2]
        
        # normality assessment
        normal <- (shapiro.test(resid(alt_model))$p.value > 0.05)
    
        # homogeneity of variance, H0: equal
        # https://ademos.people.uic.edu/Chapter18.html#62_assumption_2_homogeneity_of_variance
        clr_data$sq.resid <- abs(resid(alt_model))^2
        levene <- lm(sq.resid ~ Site, data = clr_data)
        equal_var <- (anova(levene)[1,5] > 0.05)

        
    } 
    
    # place all information into data frame
    df <- data.frame(feature,main_p_val, Lowest_AIC, null_comp, main_slope, normal, equal_var)
    rownames(df) <- NULL
    names(df) <- names(results)

    results %<>% bind_rows(df)
}

t.end <- Sys.time()
t.end - t.start

In [ ]:
non_norm <- results %>% filter(Normal_Dist == FALSE)
unequal_vars <- results %>% filter(Equal_var == FALSE)

In [ ]:
n_features <- nrow(non_norm)
                  
clr_data <- dplyr::select(Sample, all_of(non_norm$Feature)) %>% 
    mutate(across(all_of(non_norm$Feature), function(x) if_else(x<0, -1*(abs(x)^(1/3)), x^(1/3)))) %>% 
    left_join(metadata) %>% 
    as.data.frame() %>% 
    relocate(Sample)
                  
H_null <- 'time_point'
H_alt <- 'x.scaled + time_point'
rand <- 'mouse_ID'                 
                  

# --------------------- Linear Mixed Effects Modeling ---------------------
# Using this method to assess differential abundance while maintaining power

t.start <- Sys.time()

# instantiate data frame to return results in
xformed <- data.frame(Feature = character(),
                      Main_p_value = double(),
                      Lowest_AIC = logical(),
                      Sig_to_H0 = logical(),
                      Main_Slope = double(),
                      Normal_Dist = logical(),
                      Equal_var = logical(),
                      stringsAsFactors = FALSE)


# loop over features and get models!
for (i in 2:n_features){
    
    feature <- colnames(clr_data[i])
    fixed.null <- formula(paste(feature,'~',H_null))
    fixed.alt <- formula(paste(feature,'~',H_alt))
    rando.form <- formula(paste('~ 1|',rand, sep=''))
    
    tryCatch(
        # TRY BLOCK
        { 
            # fitting the linear mixed effects model
            # null model
            {null_model <- lme(data = clr_data,
                      fixed = fixed.null,
                      random = rando.form,
                      na.action=na.omit,
                      method='ML')}
            
            {alt_model <- lme(data = clr_data,
                     fixed = fixed.alt,
                     random = rando.form,
                     na.action=na.omit,
                     method='ML')}
        },
        # ERROR BLOCK
        error=function(e) {
            null_model <- NULL
            alt_model <- NULL
       })

    if(!is.null(alt_model)){
        # compare LMMs and get T/F lowest AIC
        comparison <- anova(null_model, alt_model)
        Lowest_AIC <- (which.min(comparison$AIC) == 2) 
        null_comp <- (comparison[2,9] <= 0.05)
        
        # get p-values 
        test <- car::Anova(alt_model, type = 2)
        main_p_val <- test[1,3]
        
        # get coefficients
        main_slope <- coef(alt_model)[1,2]
        
        # normality assessment
        normal <- (shapiro.test(resid(alt_model))$p.value > 0.05)
    
        # homogeneity of variance, H0: equal
        # https://ademos.people.uic.edu/Chapter18.html#62_assumption_2_homogeneity_of_variance
        clr_data$sq.resid <- abs(resid(alt_model))^2
        levene <- lm(sq.resid ~ Site, data = clr_data)
        equal_var <- (anova(levene)[1,5] > 0.05)
        
    } 
    
    # place all information into data frame
    df <- data.frame(feature,main_p_val, Lowest_AIC, null_comp, main_slope, normal, equal_var)
    rownames(df) <- NULL
    names(df) <- names(xformed)

    xformed %<>% bind_rows(df)
}

t.end <- Sys.time()
t.end - t.start

In [ ]:
n_features <- nrow(unequal_vars)

clr_data <- dplyr::select(Sample, all_of(unequal_vars$Feature)) %>% 
    left_join(metadata) %>% 
    as.data.frame() %>% 
    relocate(Sample)
                  
H_null <- 'time_point'
H_alt <- 'x.scaled + time_point'
rand <- 'mouse_ID'                   
                  
                  
# --------------------- Linear Mixed Effects Modeling ---------------------
# Using this method to assess differential abundance while maintaining power

t.start <- Sys.time()

# instantiate data frame to return results in
weighted <- data.frame(Feature = character(),
                       Main_p_value = double(),
                       Lowest_AIC = logical(),
                       Sig_to_H0 = logical(),
                       Main_Slope = double(),
                       Normal_Dist = logical(),
                       Equal_var = logical(),
                       stringsAsFactors = FALSE)


# loop over features and get models!
for (i in 2:n_features){
    
    feature <- colnames(clr_data[i])
    fixed.null <- formula(paste(feature,'~',H_null))
    fixed.alt <- formula(paste(feature,'~',H_alt))
    rando.form <- formula(paste('~ 1|',rand, sep=''))
    weight <- formula(paste('~I(1/',feature,')'))
    
    tryCatch(
        # TRY BLOCK
        { 
            # fitting the linear mixed effects model
            # null model
            {null_model <- lme(data = clr_data,
                      fixed = fixed.null,
                      random = rando.form,
                      na.action=na.omit,
                      method='ML',
                      weights = weight)}
            
            {alt_model <- lme(data = clr_data,
                     fixed = fixed.alt,
                     random = rando.form,
                     na.action=na.omit,
                     method='ML',
                     weights = weight)}
        },
        # ERROR BLOCK
        error=function(e) {
            null_model <- NULL
            alt_model <- NULL
       })

    if(!is.null(alt_model)){
        # compare LMMs and get T/F lowest AIC
        comparison <- anova(null_model, alt_model)
        Lowest_AIC <- (which.min(comparison$AIC) == 2) 
        null_comp <- (comparison[2,9] <= 0.05)
        
        # get p-values 
        test <- car::Anova(alt_model, type = 2)
        main_p_val <- test[1,3]
        
        # get coefficients
        main_slope <- coef(alt_model)[1,2]
        
        # normality assessment
        normal <- (shapiro.test(resid(alt_model))$p.value > 0.05)
    
        # homogeneity of variance, H0: equal
        # https://ademos.people.uic.edu/Chapter18.html#62_assumption_2_homogeneity_of_variance
        clr_data$sq.resid <- abs(resid(alt_model))^2
        levene <- lm(sq.resid ~ Site, data = clr_data)
        equal_var <- (anova(levene)[1,5] > 0.05)
        
    } 
    
    # place all information into data frame
    df <- data.frame(feature, main_p_val, Lowest_AIC, null_comp, main_slope, normal, equal_var)
    rownames(df) <- NULL
    names(df) <- names(weighted)

    weighted %<>% bind_rows(df)
}

t.end <- Sys.time()
t.end - t.start

In [ ]:
results %<>% bind_rows(xformed, weighted)

lmms <- results %>% 
    filter(Lowest_AIC==T, Sig_to_H0==T, Normal_Dist==T, Equal_var==T) %>% 
    group_by(Feature) %>% 
    filter(Main_p_value == min(Main_p_value)) %>% 
    ungroup() %>% 
    mutate(main_p_adj = p.adjust(Main_p_value, method='fdr'), 
           covar_p_adj = p.adjust(Covar_p_value, method='fdr')) %>% 
    filter(main_p_adj <= 0.05)

In [ ]:
# save to file
lmms %>% write_csv(paste(lubridate::today(),'LMM_Continuous_Variable.csv', sep='_'))

In [ ]:
# What is changing the most? 
age <- lmms %>% 
    filter(main_p_adj <= 0.05) %>% 
    arrange(Main_Slope) 

higher <- x %>% slice_head(n=10)
lower <- x %>% slice_tail(n=10)
most_diff <- bind_rows(higher, lower)

data <- clr_counts %>% 
    dplyr::select(Sample, all_of(most_diff$Feature)) %>% 
    pivot_longer(-Sample, names_to='code', values_to='n') %>% 
    left_join(most_diff, by=c('code' = 'Feature')) %>% 
    left_join(Feature_code) %>% 
    left_join(metadata)

In [ ]:
data %>% 
    mutate(Species = fct_reorder(Feature, Main_Slope)) %>% 
    ggplot(aes(factor(x), Feature)) +
    geom_tile(aes(fill = n)) +
    scale_fill_gradient2(name="CLR\nAbundance", low='#4E84C4', mid='#F4EDCA', high='#D16103') +
    theme_minimal(base_size=16) +
    labs(title = 'Significantly Changing Features',
         x = 'Continuous Variable',
         y = NULL) +
    theme(plot.title = element_text(hjust = 0.5),
          panel.grid.major = element_blank(), 
          panel.grid.minor = element_blank()) 

In [ ]:
ggsave(filename=paste(lubridate::today(),'Continuous Variable.pdf'),
       dpi='retina', device='pdf', bg='transparent')